In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [206]:
data_path = r"E:\2nd year\Winter project\Thyroid Performance Analysis\Data"

In [2]:
data_path = r"C:\Users\dwija\OneDrive\Thyroid-Performance-Analysis\Data"

In [3]:
ann_test = pd.read_csv(data_path+r'\anntest.CSV')
ann_train = pd.read_csv(data_path+r'\anntrain.CSV')
hyper_test = pd.read_csv(data_path+r'\hyperTest.CSV')
hyper_train = pd.read_csv(data_path+r'\hyperTrain.CSV')
hypo_test = pd.read_csv(data_path+r'\hypoTest.CSV')
hypo_train = pd.read_csv(data_path+r'\hypoTrain.CSV')
all_hypo_train = pd.read_csv(data_path+r'\hypothyroid.csv')
euthyroid = pd.read_csv(data_path+r'\sick-euthyroid.CSV')
thyroid0387 = pd.read_csv(data_path+r'\thyroid0387EDIT.CSV')

In [4]:
data = {
    'ann test': ann_test,
    'ann train': ann_train,
    'hyper test': hyper_test,
    'hyper train': hyper_train,
    'hypo test': hypo_test,
    'hypo train': hypo_train,
}


In [5]:
bl_start = "\033[1;6m"
bl_end = "\033[0m"
for df_name, df in data.items():
    print("\n")
    print(f"{bl_start}{df_name} columns{bl_end} :\n")
    print(df.dtypes)
    print("\n")



ann test columns :

age                          float64
sex                            int64
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment                 int64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                  int64
psych                          int64
TSH                          float64
T3                           float64
TT4                          float64
T4U                          float64
FTI                          float64
Target                         int64
dtype: object




ann train columns :

age                          float64
sex                            int64
on_thyroxine                   int64
query_on_thyro

In [6]:
for df_name, df in data.items():
    print("\n")
    print(f"{bl_start}{df_name} null columns{bl_end}  :\n")
    print(df.isnull().sum())
    
    print("\n")



ann test null columns  :

age                          0
sex                          0
on_thyroxine                 0
query_on_thyroxine           0
on_antithyroid_medication    0
sick                         0
pregnant                     0
thyroid_surgery              0
I131_treatment               0
query_hypothyroid            0
query_hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
TSH                          0
T3                           0
TT4                          0
T4U                          0
FTI                          0
Target                       0
dtype: int64




ann train null columns  :

age                          0
sex                          0
on_thyroxine                 0
query_on_thyroxine           0
on_antithyroid_medication    0
sick                         0
pregnant                     0
thyroid_surgery              

In [7]:
for df_name, df in data.items():
    print(df_name, "duplicate rows :",df.duplicated().sum())


ann test duplicate rows : 8
ann train duplicate rows : 63
hyper test duplicate rows : 0
hyper train duplicate rows : 0
hypo test duplicate rows : 0
hypo train duplicate rows : 0


In [8]:
ann_test[ann_test.duplicated()]

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,goitre,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,Target
1034,0.26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00232,0.0201,0.11329,0.096,0.11776,3
1520,0.39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00232,0.0201,0.11329,0.096,0.11776,3
2031,0.39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00232,0.0201,0.11329,0.096,0.11776,3
2307,0.51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00232,0.0201,0.11329,0.096,0.11776,3
2680,0.39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00232,0.0201,0.11329,0.096,0.11776,3
2682,0.34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00232,0.0201,0.11329,0.096,0.11776,3
2730,0.50,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00232,0.0201,0.11329,0.096,0.11776,3
3369,0.36,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00232,0.0201,0.11329,0.096,0.11776,3


In [9]:
ann_train[ann_train.duplicated()]

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,goitre,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,Target
267,0.35,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3
323,0.32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3
336,0.26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3
388,0.32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3
419,0.32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3218,0.57,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3
3269,0.57,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3
3414,0.54,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3
3520,0.34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00189,0.0206,0.11118,0.099,0.11207,3


In [10]:
ann_train = ann_train.drop_duplicates()
ann_test = ann_test.drop_duplicates()

In [11]:
print("ann test duplicate rows : ",ann_test.duplicated().sum())
print("ann test duplicate rows : ",ann_train.duplicated().sum())



ann test duplicate rows :  0
ann test duplicate rows :  0


In [27]:
for df_name, df in data.items():
    # Check if 'ID' column exists before trying to drop it
    if 'ID' in df.columns:
        df.drop("ID", axis=1, inplace=True)

In [28]:
def notCorrect_TargetFilter(df,correct_Target,target):
    df = df[df.Target.isin(correct_Target)]
    df.replace(correct_Target,target,inplace = True)
    return df
    
hyper_test = notCorrect_TargetFilter(hyper_test,["hyperthyroid","T3_toxic","goitre","secondary_toxic"],"hyperthyroid")
hyper_train = notCorrect_TargetFilter(hyper_train,["hyperthyroid","T3_toxic","goitre","secondary_toxic"],"hyperthyroid")
hypo_test = notCorrect_TargetFilter(hypo_test,["hypothyroid", "primary_hypothyroid", "compensated_hypothyroid", "secondary_hypothyroid"],"hypothyroid")
hypo_train = notCorrect_TargetFilter(hypo_train,["hypothyroid", "primary_hypothyroid", "compensated_hypothyroid", "secondary_hypothyroid"],"hypothyroid")

In [29]:
hypo_hyper_data = {
    'hyper test': hyper_test,
    'hyper train': hyper_train,
    'hypo test': hypo_test,
    'hypo train': hypo_train,
}

In [30]:
Dataset = pd.concat([hyper_test,hyper_train,hypo_test,hypo_train], ignore_index = True)
display(Dataset.shape)

(393, 30)

In [32]:
Dataset['sex'] = Dataset['sex'].replace({'M': 0, 'F': 1})
display(Dataset)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Target
0,79,0,f,f,f,f,f,f,f,f,...,t,151,t,0.74,t,203,f,?,other,hyperthyroid
1,50,1,f,f,t,f,f,f,f,f,...,t,131,t,1.02,t,129,f,?,other,hyperthyroid
2,63,?,f,f,f,f,f,f,f,f,...,t,162,t,0.81,t,200,f,?,other,hyperthyroid
3,58,1,f,f,f,f,f,f,f,t,...,t,256,t,0.9,t,283,f,?,other,hyperthyroid
4,39,1,f,f,f,f,f,f,f,f,...,t,209,t,1.06,t,197,f,?,other,hyperthyroid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,60,1,f,f,f,f,f,f,f,t,...,t,45,t,0.97,t,46,f,?,other,hypothyroid
389,67,1,f,f,f,f,f,f,f,f,...,t,48,t,0.98,t,49,f,?,SVI,hypothyroid
390,46,1,f,f,f,f,f,f,f,f,...,t,48,t,0.89,t,54,f,?,other,hypothyroid
391,24,1,f,f,f,f,f,f,f,f,...,t,110,t,1.17,t,94,f,?,other,hypothyroid


In [33]:
Dataset.replace('?', np.nan, inplace=True)
display(Dataset)

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4_measured,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Target
0,79,0.0,f,f,f,f,f,f,f,f,...,t,151,t,0.74,t,203,f,NaN,other,hyperthyroid
1,50,1.0,f,f,t,f,f,f,f,f,...,t,131,t,1.02,t,129,f,NaN,other,hyperthyroid
2,63,NaN,f,f,f,f,f,f,f,f,...,t,162,t,0.81,t,200,f,NaN,other,hyperthyroid
3,58,1.0,f,f,f,f,f,f,f,t,...,t,256,t,0.9,t,283,f,NaN,other,hyperthyroid
4,39,1.0,f,f,f,f,f,f,f,f,...,t,209,t,1.06,t,197,f,NaN,other,hyperthyroid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,60,1.0,f,f,f,f,f,f,f,t,...,t,45,t,0.97,t,46,f,NaN,other,hypothyroid
389,67,1.0,f,f,f,f,f,f,f,f,...,t,48,t,0.98,t,49,f,NaN,SVI,hypothyroid
390,46,1.0,f,f,f,f,f,f,f,f,...,t,48,t,0.89,t,54,f,NaN,other,hypothyroid
391,24,1.0,f,f,f,f,f,f,f,f,...,t,110,t,1.17,t,94,f,NaN,other,hypothyroid


In [ ]:
Dataset = Dataset.drop(['TBG', 'TBG Measured'], axis=1)


In [39]:
Dataset = Dataset.dropna(thresh=Dataset.shape[1] - 5)
nan_counts = Dataset.isnull().sum()
nan_threshold = 0.2  # Adjust the threshold as needed
columns_to_drop = nan_counts[nan_counts > len(Dataset) * nan_threshold].index
nan_threshold = 0.2  # Adjust the threshold as needed
columns_to_drop = nan_counts[nan_counts > len(Dataset) * nan_threshold].index
Dataset = Dataset.drop(columns=columns_to_drop)
Dataset1 = Dataset.interpolate(method = 'spline', order = 3)
display(Dataset1.isna().sum())


age                           0
sex                           0
on_thyroxine                  0
query_on_thyroxine            0
on_antithyroid_medication     0
sick                          0
pregnant                      0
thyroid_surgery               0
I131_treatment                0
query_hypothyroid             0
query_hyperthyroid            0
lithium                       0
goitre                        0
tumor                         0
hypopituitary                 0
psych                         0
TSH_measured                  0
TSH                          13
T3_measured                   0
T3                           59
TT4_measured                  0
TT4                           5
T4U_measured                  0
T4U                          26
FTI_measured                  0
FTI                          26
referral_source               0
Target                        0
dtype: int64

In [161]:
for df_name, df in data.items():
    print(f"{bl_start}Detailed Statistics of {df_name} {bl_end}  :\n")
    print(df.info(),"\n")

Detailed Statistics of ann test   :

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3428 entries, 0 to 3427
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3428 non-null   float64
 1   sex                        3428 non-null   int64  
 2   on_thyroxine               3428 non-null   int64  
 3   query_on_thyroxine         3428 non-null   int64  
 4   on_antithyroid_medication  3428 non-null   int64  
 5   sick                       3428 non-null   int64  
 6   pregnant                   3428 non-null   int64  
 7   thyroid_surgery            3428 non-null   int64  
 8   I131_treatment             3428 non-null   int64  
 9   query_hypothyroid          3428 non-null   int64  
 10  query_hyperthyroid         3428 non-null   int64  
 11  lithium                    3428 non-null   int64  
 12  goitre                     3428 non-null   int64  
 13  tumor      

In [162]:
for df_name, df in data.items():
    print(f"{bl_start}Summary Statistics of {df_name} {bl_end}  :\n")
    print(df.describe(),"\n")


Summary Statistics of ann test   :

               age          sex  on_thyroxine  query_on_thyroxine  \
count  3428.000000  3428.000000   3428.000000         3428.000000   
mean      0.526478     0.305718      0.138565            0.018086   
std       0.188836     0.460778      0.345542            0.133283   
min       0.010000     0.000000      0.000000            0.000000   
25%       0.370000     0.000000      0.000000            0.000000   
50%       0.550000     0.000000      0.000000            0.000000   
75%       0.680000     1.000000      0.000000            0.000000   
max       0.970000     1.000000      1.000000            1.000000   

       on_antithyroid_medication         sick     pregnant  thyroid_surgery  \
count                3428.000000  3428.000000  3428.000000      3428.000000   
mean                    0.014294     0.038215     0.007293         0.013711   
std                     0.118717     0.191742     0.085099         0.116304   
min                     0.

In [163]:
ann_data = {
    'ann test': ann_test,
    'ann train': ann_train,
}


In [165]:
hypo_hyper_data = {
    'hyper test': hyper_test,
    'hyper train': hyper_train,
    'hypo test': hypo_test,
    'hypo train': hypo_train,
}